# Задание 3

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [122]:
import inspect
import functools

In [123]:
class LruCache(object):
    def __init__(self, max_count):
        self.max_count = max_count
        self.cache = {}

    def __getitem__(self, key):
        if key not in self.cache:
            return None
        
        value = self.cache[key]
        del self.cache[key]
        self.cache[key] = value
        return value

    def __setitem__(self, key, value):
        if key in self.cache:
            del self.cache[key]
        self.cache[key] = value
        
        if len(self.cache) > self.max_count:
            del self.cache[next(iter(self.cache))]


In [124]:
lru = LruCache(2)
lru[1] = 1
lru[2] = 2
# print(lru[1])
lru[1] = 2
lru[3] = 3
print(lru[1], lru[2], lru[3])

2 None 3


#### Реализуйте декоратор

In [125]:
# хотим использовать в качестве ключей словаря
# поэтому надо сделать неизменяемым
# считается что в качестве значений могут быть другие словари
def dict_to_tuple(d):
    for k in d:
        if type(d[k]) is dict:
            d[k] = dict_to_tuple(d[k])
    return tuple(d.items())

def cached(max_count, verbose=False):
    def decorator(func):
        cache = LruCache(max_count)
        signature = inspect.signature(func)
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            bound = signature.bind(*args, **kwargs)    
            bound_args = dict_to_tuple(bound.arguments)
#             if verbose:
#                 print(bound_args)
            
            cached_result = cache[bound_args]
            if cached_result is not None:
                if verbose:
                    print('From cache')
                return cached_result
            
            if verbose:
                print('Evaluating')
            result = func(*args, **kwargs)
            cache[bound_args] = result
            return result
            
        return wrapper
        
    return decorator

#### Проверьте использование декоратора

In [126]:
@cached(20, verbose=True)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

@cached(20, verbose=True)
def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

In [127]:
print(fact(5))
print(fact(6))

Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
120
Evaluating
From cache
720


In [128]:
print(fact1(7))
print(fact1(21))

Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
5040
Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
Evaluating
From cache
51090942171709440000


#### Сравните свою реализацию с lru_cache из functools

In [129]:
from functools import lru_cache

@cached(10)
def foo(a, b, c):
    print('Evaluating')
    return a + b + c
    
@lru_cache(maxsize=10)    
def foo1(a, b, c):
    print('Evaluating')
    return a + b + c

In [130]:
print(foo(1, 2, 3))
print('again')
print(foo(1, 2, 3))

Evaluating
6
again
6


In [131]:
print(foo1(1, 2, 3))
print('again')
print(foo1(1, 2, 3))

Evaluating
6
again
6


In [132]:
print(foo(3, 2, c=2))
print('again')
print(foo(3, 2, 2))

Evaluating
7
again
7


In [133]:
print(foo1(3, 2, c=2))
print('again')
print(foo1(3, 2, 2))

Evaluating
7
again
Evaluating
7


Реализация из `functools` не смогла вернуть закэшированное значение, если в первый раз параметр передаётся как positional, а во второй раз -- как keyword. 

### Дополнительное задание (0.2 балла)

Дополните декоратор @cached так, чтобы не пересчитывать функцию при изменения ее состояния (например, она использовала глобальную переменную)

In [ ]:
<your code here>

## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [177]:
def checked(*types):
    def decorator(func):
        assert len(types) == func.__code__.co_argcount
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            assert len(types) == len(args), \
                    'неверное количество аргументов'
            for (a, t) in zip(args, types):
                if not isinstance(a, t):
                    raise TypeError(f'''Неправильный тип аргумента {a}: \
ожидался {t}, получен {type(a)}''')
            return func(*args, **kwargs)
        return wrapper
    return decorator

#### Проверьте использование декоратора

In [178]:
from typing import List

# Пример
@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    pass

In [179]:
# всё ок
strange_func('a', 1, [1, 'b'])

In [180]:
strange_func(1, 2, 3, 4)

AssertionError: неверное количество аргументов

In [181]:
strange_func('a', 1, {'a' : 2})

TypeError: Неправильный тип аргумента {'a': 2}: ожидался <class 'list'>, получен <class 'dict'>

In [182]:
strange_func(1, 'a', [])

TypeError: Неправильный тип аргумента 1: ожидался <class 'str'>, получен <class 'int'>

## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

Я не совсем понял, как реализовать Logger, чтобы он одновременно был классом-декоратором, и его экземпляр создавался бы всего один раз для всех функций. Поэтому вручную создаётся один экземпляр Logger, который уже и передаётся в декоратор.

In [161]:
import sys
import datetime

In [162]:
class Logger:
    def __init__(self, file=sys.stdout):
        self.file = file
        self.index = 0
    
    def log(self, func_name, args, kwargs, result):
        string = f'{self.index} {datetime.datetime.now()} '
        string += f'{func_name} {args} {kwargs} {result}\n'
        if self.file == sys.stdout or self.file == sys.stderr:
            self.file.write(string)
        else:
            with open(self.file, 'a') as f:
                f.write(string)
                
        self.index += 1
                

In [163]:
def logger(Logger_inst):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            Logger_inst.log(func.__name__, args, kwargs, result)
            return result
        return wrapper
    return decorator

Можно также было создавать статический экземпляр Logger в функции logger.

In [164]:
L = Logger(sys.stderr)

In [165]:
@logger(L)
def foo(a=1, b=2, *args, **kwargs):
    return a + b

In [166]:
foo(2, 2, 'a', 'b', k=3, z=4)

0 2020-03-11 22:37:50.849691 foo (2, 2, 'a', 'b') {'k': 3, 'z': 4} 4


4

In [167]:
@logger(L)
def bar(x):
    return x

In [168]:
bar(1)

1 2020-03-11 22:37:51.656271 bar (1,) {} 1


1

In [170]:
foo(bar(1), 1)

2 2020-03-11 22:38:07.325343 bar (1,) {} 1
3 2020-03-11 22:38:07.326341 foo (1, 1) {} 2


2